# Assignment 3.1: Feature Store - Exercise 

## Import libaries

In [2]:
import boto3
import sagemaker

original_boto3_version = boto3.__version__
%pip install 'boto3>1.17.21'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sagemaker.session import Session

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

In [4]:
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "sagemaker-featurestore-assignment"

print(default_s3_bucket_name)

sagemaker-us-east-1-652903355321


In [5]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print(role)

arn:aws:iam::652903355321:role/LabRole


## Import dataset to S3

In [6]:
import os
# current working directory
current_directory = os.getcwd()
# elative path to the data file within the current directory
file_path1 = os.path.join(current_directory, 'housing_gmaps_data_raw.csv')
file_path2 = os.path.join(current_directory, 'housing.csv')
print("Path of the data file:", file_path1)
print("Path of the data file:", file_path2)

Path of the data file: /root/MLOPSAssignments/housing_gmaps_data_raw.csv
Path of the data file: /root/MLOPSAssignments/housing.csv


In [7]:
s3 = boto3.resource('s3')
s3.Object(default_s3_bucket_name, 'housingdata/csv/housing_gmaps_data_raw.csv').put(Body=open(file_path1, 'rb'))
s3.Object(default_s3_bucket_name, 'housingdata/csv/housing.csv').put(Body=open(file_path2, 'rb'))

{'ResponseMetadata': {'RequestId': 'RSJ8GGH9X1QPPWAM',
  'HostId': 'bSCkBJ173jvY46LU1K5Y+iMPlzqhShLOHxXw+/cgUG0FQGRjPpUVxV0CCc7t5HsEwEy2zZIR93Z5dlm/spO3lg==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'bSCkBJ173jvY46LU1K5Y+iMPlzqhShLOHxXw+/cgUG0FQGRjPpUVxV0CCc7t5HsEwEy2zZIR93Z5dlm/spO3lg==',
   'x-amz-request-id': 'RSJ8GGH9X1QPPWAM',
   'date': 'Sun, 19 May 2024 20:59:48 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d1c47305887e2252bf1ccbd74ff159a6"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d1c47305887e2252bf1ccbd74ff159a6"',
 'ServerSideEncryption': 'AES256'}

## Read the dataset

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io

s3_client = boto3.client("s3", region_name=region)

housing_data_bucket_name = default_s3_bucket_name
housing_gmaps_file_key = (
    "housingdata/csv/housing_gmaps_data_raw.csv"
)
housing_file_key = (
    "housingdata/csv/housing.csv"
)

housing_data_object = s3_client.get_object(
    Bucket=housing_data_bucket_name, Key=housing_file_key
)
housing_gmaps_data_object = s3_client.get_object(
    Bucket=housing_data_bucket_name, Key=housing_gmaps_file_key
)

housing_data = pd.read_csv(io.BytesIO(housing_data_object["Body"].read()))
housing_gmaps_data = pd.read_csv(io.BytesIO(housing_gmaps_data_object["Body"].read()))


In [9]:
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [10]:
housing_gmaps_data.head()

,street_number,route,locality-political,administrative_area_level_2-political,administrative_area_level_1-political,country-political,postal_code,address,longitude,latitude,...,establishment-natural_feature,airport-establishment-point_of_interest,political-sublocality-sublocality_level_1,administrative_area_level_3-political,post_box,establishment-light_rail_station-point_of_interest-transit_station,establishment-point_of_interest,aquarium-establishment-park-point_of_interest-tourist_attraction-zoo,campground-establishment-lodging-park-point_of_interest-rv_park-tourist_attraction,cemetery-establishment-park-point_of_interest
0,3130,Grizzly Peak Boulevard,Berkeley,Alameda County,California,United States,94705.0,"3130 Grizzly Peak Blvd, Berkeley, CA 94705, USA",-122.23,37.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,Tunnel Road,Oakland,Alameda County,California,United States,94611.0,"2005 Tunnel Rd, Oakland, CA 94611, USA",-122.22,37.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6886,Chabot Road,Oakland,Alameda County,California,United States,94618.0,"6886 Chabot Rd, Oakland, CA 94618, USA",-122.24,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6365,Florio Street,Oakland,Alameda County,California,United States,94618.0,"6365 Florio St, Oakland, CA 94618, USA",-122.25,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5407,Bryant Avenue,Oakland,Alameda County,California,United States,94618.0,"5407 Bryant Ave, Oakland, CA 94618, USA",-122.25,37.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Address null values in housing_gmaps_data_raw dataset

## 1. Addres null values in 'postal_code' column

In [11]:
null_postal_count = housing_gmaps_data['postal_code'].isna().sum()
print("count of null values in postal_code column", null_postal_count)

count of null values in postal_code column 180


##### Update null values in postal_code by most frequent values of postal_code by administrative_area_level_2-political

In [12]:
housing_gmaps_data['postal_code'] = housing_gmaps_data.groupby('administrative_area_level_2-political')['postal_code'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
null_postal_afterprocess_count = housing_gmaps_data['postal_code'].isna().sum()
print("count of null values in postal_code column", null_postal_afterprocess_count)

count of null values in postal_code column 47


##### Update null values in postal_code by most frequent values of postal_code by administrative_area_level_1-political

In [13]:
housing_gmaps_data['postal_code'] = housing_gmaps_data.groupby('administrative_area_level_1-political')['postal_code'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
null_postal_afterprocess_count = housing_gmaps_data['postal_code'].isna().sum()
print("count of null values in postal_code column", null_postal_afterprocess_count)

count of null values in postal_code column 4


##### Update remaing null values in postal_code by most frequent values of postal_code in dataset

In [14]:
housing_gmaps_data['postal_code']=housing_gmaps_data['postal_code'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
null_postal_afterprocess_count = housing_gmaps_data['postal_code'].isna().sum()
print("count of null values in postal_code column", null_postal_afterprocess_count)

count of null values in postal_code column 0


## 2. Address null values in 'locality-political' column. Impute missing values by getting average for locality-code

In [15]:
null_locality_count = housing_gmaps_data['locality-political'].isna().sum()
print("Count of null values in locality-political column is", null_locality_count)

Count of null values in locality-political column is 187


##### Create locality-code column by ordinal elocality-political

In [16]:
# Initialize OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder
df = pd.DataFrame(housing_gmaps_data['locality-political'])
encoder = OrdinalEncoder()
encoded_locality_political=encoder.fit_transform(df[['locality-political']])
encoded_locality_political= pd.DataFrame(encoded_locality_political, columns=['locality-code'])

housing_gmaps_data = pd.concat(
    [housing_gmaps_data, encoded_locality_political], axis=1
)
housing_gmaps_data.head(5)

,street_number,route,locality-political,administrative_area_level_2-political,administrative_area_level_1-political,country-political,postal_code,address,longitude,latitude,...,airport-establishment-point_of_interest,political-sublocality-sublocality_level_1,administrative_area_level_3-political,post_box,establishment-light_rail_station-point_of_interest-transit_station,establishment-point_of_interest,aquarium-establishment-park-point_of_interest-tourist_attraction-zoo,campground-establishment-lodging-park-point_of_interest-rv_park-tourist_attraction,cemetery-establishment-park-point_of_interest,locality-code
0,3130,Grizzly Peak Boulevard,Berkeley,Alameda County,California,United States,94705.0,"3130 Grizzly Peak Blvd, Berkeley, CA 94705, USA",-122.23,37.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0
1,2005,Tunnel Road,Oakland,Alameda County,California,United States,94611.0,"2005 Tunnel Rd, Oakland, CA 94611, USA",-122.22,37.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625.0
2,6886,Chabot Road,Oakland,Alameda County,California,United States,94618.0,"6886 Chabot Rd, Oakland, CA 94618, USA",-122.24,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625.0
3,6365,Florio Street,Oakland,Alameda County,California,United States,94618.0,"6365 Florio St, Oakland, CA 94618, USA",-122.25,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625.0
4,5407,Bryant Avenue,Oakland,Alameda County,California,United States,94618.0,"5407 Bryant Ave, Oakland, CA 94618, USA",-122.25,37.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625.0


##### Calculate average value of locality-code  

In [17]:
#Calculate average value of locality-code 
average_locality = housing_gmaps_data['locality-code'].mean().round(0)
print(average_locality)
df=housing_gmaps_data['locality-political'][housing_gmaps_data['locality-code'] == average_locality]
average_locality_value=df.iloc[0]
print(average_locality_value)

543.0
Meadow Vista


##### impute missing values with average value

In [18]:
housing_gmaps_data['locality-code'].fillna(average_locality, inplace=True)
housing_gmaps_data['locality-political'].fillna(average_locality_value, inplace=True)

/tmp/ipykernel_795/2491035302.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  housing_gmaps_data['locality-code'].fillna(average_locality, inplace=True)
/tmp/ipykernel_795/2491035302.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

In [19]:
#housing_gmaps_data['locality-code']=housing_gmaps_data['locality-code'].transform(lambda x: x.fillna(x.mean()) if not np.isnan(x.mean()) else x)
null_postal_afterprocess_count = housing_gmaps_data['locality-code'].isna().sum()
print("Count of null values in locality-code column is", null_postal_afterprocess_count)

Count of null values in locality-code column is 0


In [20]:
null_locality_afterprocess_count = housing_gmaps_data['locality-political'].isna().sum()
print("Count of null values in locality-political column is", null_locality_afterprocess_count)

Count of null values in locality-political column is 0


In [21]:
#for index, row in housing_gmaps_data.iterrows():
    # Check if the value in 'locality-political' is NaN
#    if pd.isna(row['locality-political']):
        # Find the corresponding 'locality-political' value based on 'locality-code'
 #       matching_value = housing_gmaps_data.loc[housing_gmaps_data['locality-code'] == row['locality-code'], 'locality-political'].iloc[0]
        # Update the value in 'locality-political'
  #      housing_gmaps_data.at[index, 'locality-political'] = matching_value

In [22]:
#housing_gmaps_data['locality-political'] = housing_gmaps_data.groupby('postal_code')['locality-political'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
#null_locality_afterprocess_count = housing_gmaps_data['locality-political'].isna().sum()
#print("Count of null values in locality-political column is", null_locality_afterprocess_count)

In [23]:
#housing_gmaps_data['locality-political']=housing_gmaps_data['locality-political'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
#null_postal_afterprocess_count = housing_gmaps_data['locality-political'].isna().sum()
#print("Count of null values in locality-political column is", null_postal_afterprocess_count)

## 3. Address null values in 'neighborhood-political' column

In [24]:
null_neighborhood_count = housing_gmaps_data['neighborhood-political'].isna().sum()
print("Count of null values in neighborhood-political column is", null_neighborhood_count)

Count of null values in neighborhood-political column is 8413


##### Update null values in neighborhood-political column by most frequent values of neighborhood-political by postal-code

In [25]:
housing_gmaps_data['neighborhood-political'] = housing_gmaps_data.groupby('postal_code')['neighborhood-political'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
null_neighborhood_afterprocess_count = housing_gmaps_data['neighborhood-political'].isna().sum()
print("Count of null values in neighborhood-political column is", null_neighborhood_afterprocess_count)

Count of null values in neighborhood-political column is 5994


##### Update null values in neighborhood-political column by most frequent values of neighborhood-political by locality-political

In [26]:
housing_gmaps_data['neighborhood-political'] = housing_gmaps_data.groupby('locality-political')['neighborhood-political'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
null_neighborhood_afterprocess_count = housing_gmaps_data['neighborhood-political'].isna().sum()
print("Count of null values in neighborhood-political column is", null_neighborhood_afterprocess_count)

Count of null values in neighborhood-political column is 4839


##### Update null values in neighborhood-political column by most frequent values of neighborhood-political by administrative_area_level_2-political

In [27]:
housing_gmaps_data['neighborhood-political'] = housing_gmaps_data.groupby('administrative_area_level_2-political')['neighborhood-political'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
null_neighborhood_afterprocess_count = housing_gmaps_data['neighborhood-political'].isna().sum()
print("Count of null values in neighborhood-political column is", null_neighborhood_afterprocess_count)

Count of null values in neighborhood-political column is 289


##### Update remaing null values in neighborhood-political by most frequent values of neighborhood-political in dataset

In [28]:
housing_gmaps_data['neighborhood-political']=housing_gmaps_data['neighborhood-political'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)
null_neighborhood_afterprocess_count = housing_gmaps_data['neighborhood-political'].isna().sum()
print("Count of null values in neighborhood-political column is", null_neighborhood_afterprocess_count)

Count of null values in neighborhood-political column is 0


In [29]:
# Merge the dataframes on longitude and latitude
housing_data_merged = pd.merge(housing_data, housing_gmaps_data, on=['longitude', 'latitude'], how='left')
housing_data_merged.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,...,airport-establishment-point_of_interest,political-sublocality-sublocality_level_1,administrative_area_level_3-political,post_box,establishment-light_rail_station-point_of_interest-transit_station,establishment-point_of_interest,aquarium-establishment-park-point_of_interest-tourist_attraction-zoo,campground-establishment-lodging-park-point_of_interest-rv_park-tourist_attraction,cemetery-establishment-park-point_of_interest,locality-code
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625.0


In [30]:
null_count = housing_data_merged['neighborhood-political'].isna().sum()
print(null_count)

0


In [31]:
#housing_data_processed=housing_data_merged[['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'neighborhood-political','street_number','route','locality-political','administrative_area_level_2-political','administrative_area_level_1-political','country-political','postal_code','address']]
housing_data_processed=housing_data_merged[['neighborhood-political','ocean_proximity', 'median_house_value','housing_median_age', 'households', 'total_bedrooms', 'locality-political','locality-code']]
housing_data_processed.rename(columns={'neighborhood-political': 'neighborhood'}, inplace=True)
housing_data_processed.head()

/tmp/ipykernel_795/2719570867.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_data_processed.rename(columns={'neighborhood-political': 'neighborhood'}, inplace=True)


,neighborhood,ocean_proximity,median_house_value,housing_median_age,households,total_bedrooms,locality-political,locality-code
0,Claremont,NEAR BAY,452600.0,41.0,126.0,129.0,Berkeley,69.0
1,Merriewood,NEAR BAY,358500.0,21.0,1138.0,1106.0,Oakland,625.0
2,Upper Rockridge,NEAR BAY,352100.0,52.0,177.0,190.0,Oakland,625.0
3,Rockridge,NEAR BAY,341300.0,52.0,219.0,235.0,Oakland,625.0
4,Rockridge,NEAR BAY,342200.0,52.0,259.0,280.0,Oakland,625.0


# Feature Engineering

## 1. Apply One hot encoding of ocean_proximity column

In [32]:
# Feature Engineering

housing_data_processed = housing_data_processed.round(5)

# Feature transformations for this dataset are applied before ingestion into FeatureStore.
# One hot encode ocean_proximity
encoded_ocean_proximity = pd.get_dummies(housing_data_processed["ocean_proximity"])

transformed_housing_data = pd.concat(
    [housing_data_processed, encoded_ocean_proximity], axis=1
)
# blank space is not allowed in feature name
transformed_housing_data = transformed_housing_data.rename(
    columns={"NEAR BAY": "NEAR_BAY","NEAR OCEAN": "NEAR_OCEAN","<1H OCEAN": "Lessthen_1H_OCEAN" }
)

In [33]:
transformed_housing_data.head()

,neighborhood,ocean_proximity,median_house_value,housing_median_age,households,total_bedrooms,locality-political,locality-code,Lessthen_1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN
0,Claremont,NEAR BAY,452600.0,41.0,126.0,129.0,Berkeley,69.0,False,False,False,True,False
1,Merriewood,NEAR BAY,358500.0,21.0,1138.0,1106.0,Oakland,625.0,False,False,False,True,False
2,Upper Rockridge,NEAR BAY,352100.0,52.0,177.0,190.0,Oakland,625.0,False,False,False,True,False
3,Rockridge,NEAR BAY,341300.0,52.0,219.0,235.0,Oakland,625.0,False,False,False,True,False
4,Rockridge,NEAR BAY,342200.0,52.0,259.0,280.0,Oakland,625.0,False,False,False,True,False


## 2. Median house value 
####    derived from median_house_value, average this value across all records for a neighborhood, cap this value at 500,000

In [34]:
transformed_housing_data['median_house_value_cal'] = transformed_housing_data.groupby('neighborhood')['median_house_value'].transform(lambda x: min(x.median(), 500000))
transformed_housing_data.head()

,neighborhood,ocean_proximity,median_house_value,housing_median_age,households,total_bedrooms,locality-political,locality-code,Lessthen_1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN,median_house_value_cal
0,Claremont,NEAR BAY,452600.0,41.0,126.0,129.0,Berkeley,69.0,False,False,False,True,False,446200.0
1,Merriewood,NEAR BAY,358500.0,21.0,1138.0,1106.0,Oakland,625.0,False,False,False,True,False,466100.0
2,Upper Rockridge,NEAR BAY,352100.0,52.0,177.0,190.0,Oakland,625.0,False,False,False,True,False,352100.0
3,Rockridge,NEAR BAY,341300.0,52.0,219.0,235.0,Oakland,625.0,False,False,False,True,False,284450.0
4,Rockridge,NEAR BAY,342200.0,52.0,259.0,280.0,Oakland,625.0,False,False,False,True,False,284450.0


## 3. Median house age
### derived from median_house_age, average this value across all records for a neighborhood, discretized by groups of 10 years i.e. 0-9, 10-19, 20-29, etc...

In [35]:
transformed_housing_data['median_house_age']=transformed_housing_data.groupby('neighborhood')['housing_median_age'].transform(lambda x: x.mean().round(0))
transformed_housing_data.head()

,neighborhood,ocean_proximity,median_house_value,housing_median_age,households,total_bedrooms,locality-political,locality-code,Lessthen_1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN,median_house_value_cal,median_house_age
0,Claremont,NEAR BAY,452600.0,41.0,126.0,129.0,Berkeley,69.0,False,False,False,True,False,446200.0,48.0
1,Merriewood,NEAR BAY,358500.0,21.0,1138.0,1106.0,Oakland,625.0,False,False,False,True,False,466100.0,42.0
2,Upper Rockridge,NEAR BAY,352100.0,52.0,177.0,190.0,Oakland,625.0,False,False,False,True,False,352100.0,49.0
3,Rockridge,NEAR BAY,341300.0,52.0,219.0,235.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0
4,Rockridge,NEAR BAY,342200.0,52.0,259.0,280.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0


In [36]:
max_age = transformed_housing_data['median_house_age'].max()
min_age = transformed_housing_data['median_house_age'].min()
bin_edges = list(range(int(min_age), int(max_age) + 11, 10))
bin_edges

[2, 12, 22, 32, 42, 52, 62]

In [37]:
bin_labels = [f"{i}-{i+9}" for i in range(int(min_age), int(max_age), 10)]
bin_labels

['2-11', '12-21', '22-31', '32-41', '42-51']

In [38]:
# Add the last bin label
bin_labels.append(f"{int(max_age)}-{int(max_age) + 9}")
bin_labels

['2-11', '12-21', '22-31', '32-41', '42-51', '52-61']

In [39]:
# Discretize the average values
transformed_housing_data['median_house_age_group'] = pd.cut(
    transformed_housing_data['median_house_age'],
    bins=bin_edges,
    labels=bin_labels,
    right=False
)

# Display the resulting DataFrame
transformed_housing_data.head()


,neighborhood,ocean_proximity,median_house_value,housing_median_age,households,total_bedrooms,locality-political,locality-code,Lessthen_1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN,median_house_value_cal,median_house_age,median_house_age_group
0,Claremont,NEAR BAY,452600.0,41.0,126.0,129.0,Berkeley,69.0,False,False,False,True,False,446200.0,48.0,42-51
1,Merriewood,NEAR BAY,358500.0,21.0,1138.0,1106.0,Oakland,625.0,False,False,False,True,False,466100.0,42.0,42-51
2,Upper Rockridge,NEAR BAY,352100.0,52.0,177.0,190.0,Oakland,625.0,False,False,False,True,False,352100.0,49.0,42-51
3,Rockridge,NEAR BAY,341300.0,52.0,219.0,235.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0,52-61
4,Rockridge,NEAR BAY,342200.0,52.0,259.0,280.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0,52-61


## 4. Total households
#### derived from households, average this value across all records for a neighborhood, null_count = transformed_housing_data['households'].isna().sum()must be an integer (round up if needed)


In [40]:
null_households_count = transformed_housing_data['households'].isna().sum()
print("count of null values in households column", null_households_count)

count of null values in households column 0


In [41]:
transformed_housing_data['total_households']=transformed_housing_data.groupby('neighborhood')['households'].transform(lambda x: np.ceil(x.mean()).astype(int))
transformed_housing_data.head()

,neighborhood,ocean_proximity,median_house_value,housing_median_age,households,total_bedrooms,locality-political,locality-code,Lessthen_1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN,median_house_value_cal,median_house_age,median_house_age_group,total_households
0,Claremont,NEAR BAY,452600.0,41.0,126.0,129.0,Berkeley,69.0,False,False,False,True,False,446200.0,48.0,42-51,377
1,Merriewood,NEAR BAY,358500.0,21.0,1138.0,1106.0,Oakland,625.0,False,False,False,True,False,466100.0,42.0,42-51,440
2,Upper Rockridge,NEAR BAY,352100.0,52.0,177.0,190.0,Oakland,625.0,False,False,False,True,False,352100.0,49.0,42-51,358
3,Rockridge,NEAR BAY,341300.0,52.0,219.0,235.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0,52-61,425
4,Rockridge,NEAR BAY,342200.0,52.0,259.0,280.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0,52-61,425


## 5.Bedrooms per household
#### derived from total_bedrooms and households

In [42]:
null_bedroom_count = transformed_housing_data['total_bedrooms'].isna().sum()
print("count of null values in total_bedrooms column", null_bedroom_count)

count of null values in total_bedrooms column 207


In [43]:
transformed_housing_data['total_bedrooms']=transformed_housing_data.groupby('neighborhood')['total_bedrooms'].transform(lambda x: x.fillna(x.mean()))
transformed_housing_data.head()

,neighborhood,ocean_proximity,median_house_value,housing_median_age,households,total_bedrooms,locality-political,locality-code,Lessthen_1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN,median_house_value_cal,median_house_age,median_house_age_group,total_households
0,Claremont,NEAR BAY,452600.0,41.0,126.0,129.0,Berkeley,69.0,False,False,False,True,False,446200.0,48.0,42-51,377
1,Merriewood,NEAR BAY,358500.0,21.0,1138.0,1106.0,Oakland,625.0,False,False,False,True,False,466100.0,42.0,42-51,440
2,Upper Rockridge,NEAR BAY,352100.0,52.0,177.0,190.0,Oakland,625.0,False,False,False,True,False,352100.0,49.0,42-51,358
3,Rockridge,NEAR BAY,341300.0,52.0,219.0,235.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0,52-61,425
4,Rockridge,NEAR BAY,342200.0,52.0,259.0,280.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0,52-61,425


In [44]:
null_bedroom_count = transformed_housing_data['total_bedrooms'].isna().sum()
print("count of null values in total_bedrooms column", null_bedroom_count)

count of null values in total_bedrooms column 4


In [45]:
transformed_housing_data['total_bedrooms']=transformed_housing_data['total_bedrooms'].transform(lambda x: x.fillna(x.mean()) if x.isnull().any() else x)
null_bedrooms_afterprocess_count = transformed_housing_data['total_bedrooms'].isna().sum()
print("count of null values in bedrooms column", null_bedrooms_afterprocess_count)

count of null values in bedrooms column 0


In [46]:
#Bedroom per household
transformed_housing_data['bedrooms_per_household']=(transformed_housing_data['total_bedrooms']/transformed_housing_data['households']).round(0)

In [47]:
transformed_housing_data.head()

,neighborhood,ocean_proximity,median_house_value,housing_median_age,households,total_bedrooms,locality-political,locality-code,Lessthen_1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN,median_house_value_cal,median_house_age,median_house_age_group,total_households,bedrooms_per_household
0,Claremont,NEAR BAY,452600.0,41.0,126.0,129.0,Berkeley,69.0,False,False,False,True,False,446200.0,48.0,42-51,377,1.0
1,Merriewood,NEAR BAY,358500.0,21.0,1138.0,1106.0,Oakland,625.0,False,False,False,True,False,466100.0,42.0,42-51,440,1.0
2,Upper Rockridge,NEAR BAY,352100.0,52.0,177.0,190.0,Oakland,625.0,False,False,False,True,False,352100.0,49.0,42-51,358,1.0
3,Rockridge,NEAR BAY,341300.0,52.0,219.0,235.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0,52-61,425,1.0
4,Rockridge,NEAR BAY,342200.0,52.0,259.0,280.0,Oakland,625.0,False,False,False,True,False,284450.0,52.0,52-61,425,1.0


In [48]:
transformed_housing_data_ingest=transformed_housing_data[['neighborhood','Lessthen_1H_OCEAN', 'INLAND','ISLAND', 'NEAR_BAY', 'NEAR_OCEAN', 'median_house_value_cal','median_house_age_group','total_households','bedrooms_per_household','locality-code']]

In [49]:
transformed_housing_data_ingest.head()

,neighborhood,Lessthen_1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN,median_house_value_cal,median_house_age_group,total_households,bedrooms_per_household,locality-code
0,Claremont,False,False,False,True,False,446200.0,42-51,377,1.0,69.0
1,Merriewood,False,False,False,True,False,466100.0,42-51,440,1.0,625.0
2,Upper Rockridge,False,False,False,True,False,352100.0,42-51,358,1.0,625.0
3,Rockridge,False,False,False,True,False,284450.0,52-61,425,1.0,625.0
4,Rockridge,False,False,False,True,False,284450.0,52-61,425,1.0,625.0


# Ingest Data Into Feature Store

In [50]:
from time import gmtime, strftime, sleep

neighborhood_feature_group_name = "neighborhood-feature-group-" + strftime("%d-%H-%M-%S", gmtime())
print(neighborhood_feature_group_name)

neighborhood-feature-group-19-20-59-55


In [51]:
from sagemaker.feature_store.feature_group import FeatureGroup

neighbourhood_feature_group = FeatureGroup(
    name=neighborhood_feature_group_name, sagemaker_session=feature_store_session
)

In [52]:
transformed_housing_data_ingest['median_house_age_group'] = transformed_housing_data_ingest['median_house_age_group'].astype("str").astype("string")

/tmp/ipykernel_795/1384087890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transformed_housing_data_ingest['median_house_age_group'] = transformed_housing_data_ingest['median_house_age_group'].astype("str").astype("string")


In [53]:
import time
#from featurestore.sdk import FeatureType

current_time_sec = int(round(time.time()))


def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")

def cast_boolean_to_string(data_frame):
    for col in data_frame:
        if data_frame[col].dtype == "bool":
            data_frame[col] = data_frame[col].replace({True: 'True', False: 'False'}).astype("string")


# cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(transformed_housing_data_ingest)

# cast object dtype to string.
cast_boolean_to_string(transformed_housing_data_ingest)

# record identifier and event time feature names
record_identifier_feature_name = "neighborhood"
event_time_feature_name = "EventTime"

# append EventTime feature
transformed_housing_data_ingest[event_time_feature_name] = pd.Series(
    [current_time_sec] * len(transformed_housing_data_ingest), dtype="float64"
)


# load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
neighbourhood_feature_group.load_feature_definitions(data_frame=transformed_housing_data_ingest)

/tmp/ipykernel_795/1484425223.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[label] = data_frame[label].astype("str").astype("string")
/tmp/ipykernel_795/1484425223.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[col] = data_frame[col].replace({True: 'True', False: 'False'}).astype("string")
/tmp/ipykernel_795/1484425223.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



[FeatureDefinition(feature_name='neighborhood', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='Lessthen_1H_OCEAN', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='INLAND', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='ISLAND', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='NEAR_BAY', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='NEAR_OCEAN', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='median_house_value_cal', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='median_house_age_group', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(featur

In [54]:
neighbourhood_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True,
)

def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")


wait_for_feature_group_creation_complete(feature_group=neighbourhood_feature_group)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup neighborhood-feature-group-19-20-59-55 successfully created.


In [55]:
neighbourhood_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:652903355321:feature-group/neighborhood-feature-group-19-20-59-55',
 'FeatureGroupName': 'neighborhood-feature-group-19-20-59-55',
 'RecordIdentifierFeatureName': 'neighborhood',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'neighborhood',
   'FeatureType': 'String'},
  {'FeatureName': 'Lessthen_1H_OCEAN', 'FeatureType': 'String'},
  {'FeatureName': 'INLAND', 'FeatureType': 'String'},
  {'FeatureName': 'ISLAND', 'FeatureType': 'String'},
  {'FeatureName': 'NEAR_BAY', 'FeatureType': 'String'},
  {'FeatureName': 'NEAR_OCEAN', 'FeatureType': 'String'},
  {'FeatureName': 'median_house_value_cal', 'FeatureType': 'Fractional'},
  {'FeatureName': 'median_house_age_group', 'FeatureType': 'String'},
  {'FeatureName': 'total_households', 'FeatureType': 'Integral'},
  {'FeatureName': 'bedrooms_per_household', 'FeatureType': 'Fractional'},
  {'FeatureName': 'locality-code', 'FeatureType': 'Fractional'},
  {'FeatureNa

In [56]:
sagemaker_client.list_feature_groups()  # use boto client to list FeatureGroups

{'FeatureGroupSummaries': [{'FeatureGroupName': 'neighborhood-feature-group-19-20-59-55',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:652903355321:feature-group/neighborhood-feature-group-19-20-59-55',
   'CreationTime': datetime.datetime(2024, 5, 19, 20, 59, 55, 681000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'neighborhood-feature-group-19-20-04-45',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:652903355321:feature-group/neighborhood-feature-group-19-20-04-45',
   'CreationTime': datetime.datetime(2024, 5, 19, 20, 26, 46, 931000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'neighborhood-feature-group-18-13-26-36',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:652903355321:feature-group/neighborhood-feature-group-18-13-26-36',
   'CreationTime': datetime.datetime(2024, 5, 18, 13, 26, 54, 453000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',

In [57]:
neighbourhood_feature_group.ingest(data_frame=transformed_housing_data_ingest, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='neighborhood-feature-group-19-20-59-55', feature_definitions={'neighborhood': {'FeatureName': 'neighborhood', 'FeatureType': 'String'}, 'Lessthen_1H_OCEAN': {'FeatureName': 'Lessthen_1H_OCEAN', 'FeatureType': 'String'}, 'INLAND': {'FeatureName': 'INLAND', 'FeatureType': 'String'}, 'ISLAND': {'FeatureName': 'ISLAND', 'FeatureType': 'String'}, 'NEAR_BAY': {'FeatureName': 'NEAR_BAY', 'FeatureType': 'String'}, 'NEAR_OCEAN': {'FeatureName': 'NEAR_OCEAN', 'FeatureType': 'String'}, 'median_house_value_cal': {'FeatureName': 'median_house_value_cal', 'FeatureType': 'Fractional'}, 'median_house_age_group': {'FeatureName': 'median_house_age_group', 'FeatureType': 'String'}, 'total_households': {'FeatureName': 'total_households', 'FeatureType': 'Integral'}, 'bedrooms_per_household': {'FeatureName': 'bedrooms_per_household', 'FeatureType': 'Fractional'}, 'locality-code': {'FeatureName': 'locality-code', 'FeatureType': 'Fractional'}, 'EventTime': {'FeatureN

In [58]:
record_identifier_value = "Merriewood"

featurestore_runtime.get_record(
    FeatureGroupName=neighborhood_feature_group_name,
    RecordIdentifierValueAsString=record_identifier_value,
)

{'ResponseMetadata': {'RequestId': 'c69453d0-86a0-475a-9a80-7d72e39f1d4d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c69453d0-86a0-475a-9a80-7d72e39f1d4d',
   'content-type': 'application/json',
   'content-length': '1018',
   'date': 'Sun, 19 May 2024 21:02:16 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'neighborhood', 'ValueAsString': 'Merriewood'},
  {'FeatureName': 'Lessthen_1H_OCEAN', 'ValueAsString': 'False'},
  {'FeatureName': 'INLAND', 'ValueAsString': 'False'},
  {'FeatureName': 'ISLAND', 'ValueAsString': 'False'},
  {'FeatureName': 'NEAR_BAY', 'ValueAsString': 'True'},
  {'FeatureName': 'NEAR_OCEAN', 'ValueAsString': 'False'},
  {'FeatureName': 'median_house_value_cal', 'ValueAsString': '466100.0'},
  {'FeatureName': 'median_house_age_group', 'ValueAsString': '42-51'},
  {'FeatureName': 'total_households', 'ValueAsString': '440'},
  {'FeatureName': 'bedrooms_per_household', 'ValueAsString': '1.0'},
  {'FeatureName': 'locality-code', 'ValueA

In [59]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

neighbourhood_feature_group_resolved_output_s3_uri = (
    neighbourhood_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)

neighbourhood_feature_group_s3_prefix = neighbourhood_feature_group_resolved_output_s3_uri.replace(
    f"s3://{default_s3_bucket_name}/", ""
)

offline_store_contents = None
while offline_store_contents is None:
    objects_in_bucket = s3_client.list_objects(
        Bucket=default_s3_bucket_name, Prefix=neighbourhood_feature_group_s3_prefix
    )
    if "Contents" in objects_in_bucket and len(objects_in_bucket["Contents"]) > 1:
        offline_store_contents = objects_in_bucket["Contents"]
    else:
        print("Waiting for data in offline store...\n")
        sleep(60)

print("Data available.")

652903355321
Waiting for data in offline store...

Waiting for data in offline store...

Waiting for data in offline store...

Waiting for data in offline store...

Data available.


In [60]:
print(neighbourhood_feature_group.as_hive_ddl())

CREATE EXTERNAL TABLE IF NOT EXISTS sagemaker_featurestore.neighborhood-feature-group-19-20-59-55 (
  neighborhood STRING
  Lessthen_1H_OCEAN STRING
  INLAND STRING
  ISLAND STRING
  NEAR_BAY STRING
  NEAR_OCEAN STRING
  median_house_value_cal FLOAT
  median_house_age_group STRING
  total_households INT
  bedrooms_per_household FLOAT
  locality-code FLOAT
  EventTime FLOAT
  write_time TIMESTAMP
  event_time TIMESTAMP
  is_deleted BOOLEAN
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
  STORED AS
  INPUTFORMAT 'parquet.hive.DeprecatedParquetInputFormat'
  OUTPUTFORMAT 'parquet.hive.DeprecatedParquetOutputFormat'
LOCATION 's3://sagemaker-us-east-1-652903355321/sagemaker-featurestore-assignment/652903355321/sagemaker/us-east-1/offline-store/neighborhood-feature-group-19-20-59-55-1716152395/data'


In [61]:
neighbourhood_query = neighbourhood_feature_group.athena_query()

neighbourhood_table = neighbourhood_query.table_name
query_string = (
    'SELECT * FROM '
    + neighbourhood_table    
    
)
print("Running " + query_string)

# run Athena query. The output is loaded to a Pandas dataframe.
# dataset = pd.DataFrame()
neighbourhood_query.run(
    query_string=query_string,
    output_location="s3://" + default_s3_bucket_name + "/" + prefix + "/query_results/",
)
neighbourhood_query.wait()
dataset = neighbourhood_query.as_dataframe()

dataset

Running SELECT * FROM neighborhood_feature_group_19_20_59_55_1716152395


,neighborhood,lessthen_1h_ocean,inland,island,near_bay,near_ocean,median_house_value_cal,median_house_age_group,total_households,bedrooms_per_household,locality-code,eventtime,write_time,api_invocation_time,is_deleted
0,Eastmont,False,False,False,True,False,94800.0,32-41,344,1.0,625.0,1.716152e+09,2024-05-19 21:05:57.839,2024-05-19 21:00:38.000,False
1,Oakland Ave - Harrison St,False,False,False,True,False,185650.0,32-41,893,1.0,625.0,1.716152e+09,2024-05-19 21:05:57.821,2024-05-19 21:00:34.000,False
2,Nipton,False,True,False,False,False,75000.0,2-11,56,1.0,543.0,1.716152e+09,2024-05-19 21:05:57.798,2024-05-19 21:00:35.000,False
3,Fruitvale,False,False,False,True,False,105600.0,42-51,238,1.0,625.0,1.716152e+09,2024-05-19 21:05:57.791,2024-05-19 21:00:36.000,False
4,Maxwell Park,False,False,False,True,False,160000.0,42-51,313,1.0,625.0,1.716152e+09,2024-05-19 21:05:57.797,2024-05-19 21:00:37.000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5256,Central Davis,False,True,False,False,False,118950.0,22-31,552,1.0,141.0,1.716152e+09,2024-05-19 21:05:57.836,2024-05-19 21:02:14.000,False
5257,Central Davis,False,True,False,False,False,118950.0,22-31,552,1.0,279.0,1.716152e+09,2024-05-19 21:05:57.836,2024-05-19 21:02:14.000,False
5258,Central Davis,False,True,False,False,False,118950.0,22-31,552,1.0,978.0,1.716152e+09,2024-05-19 21:05:57.836,2024-05-19 21:02:14.000,False
5259,Central Davis,False,True,False,False,False,118950.0,22-31,552,1.0,279.0,1.716152e+09,2024-05-19 21:05:57.836,2024-05-19 21:02:14.000,False


# Query the Feature Values:
#### Please query the feature values from your feature store:
#### 1.) Brooktree
#### 2.) Fisherman’s Wharf
#### 3.) Los Osos

In [62]:
def queryfeaturestore(search_string):
    neighbourhood_query = neighbourhood_feature_group.athena_query()

    neighbourhood_table = neighbourhood_query.table_name
    #search_string='Brooktree'
    query_string = (
        'SELECT * FROM '
        + neighbourhood_table
         + ' WHERE '
        + 'neighborhood = "'
        + search_string
        +'"'

    )

    query_string = query_string.replace('"', "'")
    print("Running " + query_string)

    # run Athena query. The output is loaded to a Pandas dataframe.
    # dataset = pd.DataFrame()
    neighbourhood_query.run(
        query_string=query_string,
        output_location="s3://" + default_s3_bucket_name + "/" + prefix + "/query_results/",
    )
    neighbourhood_query.wait()
    dataset = neighbourhood_query.as_dataframe()

    return dataset

In [63]:
search_string="Brooktree"
feature_dataset=queryfeaturestore(search_string)
feature_dataset.head()

Running SELECT * FROM neighborhood_feature_group_19_20_59_55_1716152395 WHERE neighborhood = 'Brooktree'


,neighborhood,lessthen_1h_ocean,inland,island,near_bay,near_ocean,median_house_value_cal,median_house_age_group,total_households,bedrooms_per_household,locality-code,eventtime,write_time,api_invocation_time,is_deleted
0,Brooktree,True,False,False,False,False,257400.0,2-11,1438,0.0,787.0,1.716152e+09,2024-05-19 21:05:57.793,2024-05-19 21:01:34.000,False


In [64]:
search_string="Los Osos"
feature_dataset=queryfeaturestore(search_string)
feature_dataset.head()

Running SELECT * FROM neighborhood_feature_group_19_20_59_55_1716152395 WHERE neighborhood = 'Los Osos'


,neighborhood,lessthen_1h_ocean,inland,island,near_bay,near_ocean,median_house_value_cal,median_house_age_group,total_households,bedrooms_per_household,locality-code,eventtime,write_time,api_invocation_time,is_deleted
0,Los Osos,False,False,False,False,True,196100.0,12-21,612,1.0,792.0,1.716152e+09,2024-05-19 21:05:59.658,2024-05-19 21:01:16.000,False
1,Los Osos,False,False,False,False,True,196100.0,12-21,612,1.0,55.0,1.716152e+09,2024-05-19 21:05:59.658,2024-05-19 21:01:16.000,False


In [65]:
search_string="Wharf"
neighbourhood_query = neighbourhood_feature_group.athena_query()

neighbourhood_table = neighbourhood_query.table_name
#search_string='Brooktree'
query_string = (
    'SELECT * FROM '
    + neighbourhood_table
     + ' WHERE '
    + 'neighborhood like "'
    + '% '
    + search_string
    +'"'

)

query_string = query_string.replace('"', "'")
print("Running " + query_string)

# run Athena query. The output is loaded to a Pandas dataframe.
# dataset = pd.DataFrame()
neighbourhood_query.run(
    query_string=query_string,
    output_location="s3://" + default_s3_bucket_name + "/" + prefix + "/query_results/",
)
neighbourhood_query.wait()
dataset = neighbourhood_query.as_dataframe()
dataset


Running SELECT * FROM neighborhood_feature_group_19_20_59_55_1716152395 WHERE neighborhood like '% Wharf'


,neighborhood,lessthen_1h_ocean,inland,island,near_bay,near_ocean,median_house_value_cal,median_house_age_group,total_households,bedrooms_per_household,locality-code,eventtime,write_time,api_invocation_time,is_deleted
0,Fisherman's Wharf,False,False,False,True,False,500000.0,52-61,250,1.0,781.0,1.716152e+09,2024-05-19 21:05:57.858,2024-05-19 21:00:59.000,False


In [66]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>